## Setup & Imports

In [ ]:
!pip -q install timm kaggle scikit-learn

import os, random, time
from dataclasses import dataclass
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import timm

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.utils.class_weight import compute_class_weight

## Config

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
@dataclass
class CFG:
    data_root: str = "/content/drive/MyDrive/archive (1)"
    seed: int = 42

    img_size: int = 224
    batch_size: int = 32

    model_name: str = "tf_efficientnetv2_s"

    lr_head: float = 5e-4
    lr_full: float = 1e-4
    weight_decay: float = 1e-4

    epochs_head: int = 5
    epochs_full: int = 15
    patience: int = 4

    max_other: int = 200

    use_mixup: bool = True
    mixup_alpha: float = 0.2

    focal_gamma: float = 2.0

    # threshold search
    thr_min: float = 0.20
    thr_max: float = 0.90
    thr_steps: int = 29

cfg = CFG()

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True

set_seed(cfg.seed)

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## Train-Test Split

In [ ]:
root = Path(cfg.data_root)
candidates = list(root.rglob("data"))
candidates[:5], len(candidates)

([PosixPath('/content/drive/MyDrive/archive (1)/train')], 1)

In [ ]:
train_dir = candidates[0]
test_dir = train_dir.parent / "test"
train_dir, test_dir, train_dir.exists(), test_dir.exists()

(PosixPath('/content/drive/MyDrive/archive (1)/train'),
 PosixPath('/content/drive/MyDrive/archive (1)/test'),
 True,
 True)

## Construct Class

In [ ]:

tmp_base = datasets.ImageFolder(train_dir, transform=transforms.ToTensor())
all_classes = tmp_base.classes

BMW_3 = "BMW 3 Series Sedan 2012"

BMW_5_OPTION_1 = "BMW ActiveHybrid 5 Sedan 2012"
BMW_5_OPTION_2 = "BMW M5 Sedan 2010"

BMW_7 = "BMW 7 Series Sedan 2012"

def must_exist(name):
    return name in all_classes

if not must_exist(BMW_3):
    raise RuntimeError(
        f"Não encontrei a pasta '{BMW_3}'. "
        f"Confira o dataset/pastas. Exemplo de classes BMW disponíveis: "
        f"{[c for c in all_classes if 'BMW' in c.upper()][:10]}"
    )

if must_exist(BMW_5_OPTION_1):
    BMW_5 = BMW_5_OPTION_1
    BMW_5_CHOICE = "ActiveHybrid 5"
elif must_exist(BMW_5_OPTION_2):
    BMW_5 = BMW_5_OPTION_2
    BMW_5_CHOICE = "M5"
else:
    raise RuntimeError(
        f"Não encontrei nem '{BMW_5_OPTION_1}' nem '{BMW_5_OPTION_2}'. "
        f"Classes BMW disponíveis: {[c for c in all_classes if 'BMW' in c.upper()]}"
    )

bmw7_exists = must_exist(BMW_7)
if not bmw7_exists:
    fallback_candidates = [
        "BMW X5 SUV 2007",
        "BMW X6 SUV 2012",
        "BMW Z4 Convertible 2012",
        "BMW M3 Coupe 2012",
        "BMW 3 Series Wagon 2012",
    ]
    fallback = next((c for c in fallback_candidates if must_exist(c)), None)

    msg = (
        f"Aviso: não encontrei '{BMW_7}' neste dataset.\n"
        f"Isso é esperado no Stanford Cars original.\n"
    )
    if fallback is None:
        msg += "Também não achei um fallback padrão. Listei as BMW disponíveis abaixo.\n"
        print(msg)
        print("BMW disponíveis:", [c for c in all_classes if "BMW" in c.upper()])
        raise RuntimeError("Sem classe para Série 7 e sem fallback disponível.")
    else:
        msg += f"Vou usar fallback no lugar da Série 7: '{fallback}'\n"
        print(msg)
        BMW_7 = fallback

targets = [BMW_3, BMW_5, BMW_7]

print("Targets fixas escolhidas:")
print("Classe 3:", BMW_3)
print("Classe 5:", BMW_5, f"(choice={BMW_5_CHOICE})")
print("Classe 7:", BMW_7, "(7 original)" if bmw7_exists else "(fallback)")

bmw_classes = [c for c in all_classes if "BMW" in c.upper()]
other_bmw = [c for c in bmw_classes if c not in targets]
print("Other BMW classes:", len(other_bmw))

label_map = {targets[0]: 0, targets[1]: 1, targets[2]: 2, "OTHER": 3}

idx_to_name = {0:"BMW_3Series", 1:"BMW_5Series", 2:"BMW_7Series_or_Fallback", 3:"undefined"}

⚠️ Aviso: não encontrei 'BMW 7 Series Sedan 2012' neste dataset.
Isso é esperado no Stanford Cars original.
Vou usar fallback no lugar da Série 7: 'BMW X5 SUV 2007'

✅ Targets fixas escolhidas:
Classe 3: BMW 3 Series Sedan 2012
Classe 5: BMW ActiveHybrid 5 Sedan 2012 (choice=ActiveHybrid 5)
Classe 7: BMW X5 SUV 2007 (fallback)
Other BMW classes: 10


## Dataset custom

In [ ]:
train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(cfg.img_size, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(0.2,0.2,0.2,0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

test_tfms = transforms.Compose([
    transforms.Resize(int(cfg.img_size*1.14)),
    transforms.CenterCrop(cfg.img_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

base = datasets.ImageFolder(train_dir, transform=train_tfms)
class_to_idx = base.class_to_idx

In [ ]:
class MappedSubset(Dataset):
    def __init__(self, base_ds, indices, new_labels):
        assert len(indices) == len(new_labels)
        self.base_ds = base_ds
        self.indices = list(indices)
        self.new_labels = np.array(new_labels, dtype=np.int64)

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, i):
        x, _ = self.base_ds[self.indices[i]]
        return x, int(self.new_labels[i])

base_train = datasets.ImageFolder(train_dir, transform=train_tfms)
base_val   = datasets.ImageFolder(train_dir, transform=test_tfms)

target_set = set(targets)
other_set  = set(other_bmw)

target_indices, target_labels = [], []
other_indices = []

for i, (path, y_old) in enumerate(base_train.samples):
    cls_name = base_train.classes[y_old]
    if cls_name in target_set:
        target_indices.append(i)
        target_labels.append(label_map[cls_name])
    elif cls_name in other_set:
        other_indices.append(i)

np.random.shuffle(other_indices)
MAX_OTHER = getattr(cfg, "max_other", 200)
other_indices = other_indices[:MAX_OTHER]
other_labels = [label_map["OTHER"]] * len(other_indices)

sel_idx = np.array(target_indices + other_indices)
sel_y   = np.array(target_labels + other_labels, dtype=np.int64)

print("Distribuição antes do split:")
unique, counts = np.unique(sel_y, return_counts=True)
for u, c in zip(unique, counts):
    print(idx_to_name[int(u)], c)

train_pos, val_pos = train_test_split(
    np.arange(len(sel_idx)),
    test_size=0.30,
    random_state=cfg.seed,
    stratify=sel_y
)

train_ds = MappedSubset(base_train, sel_idx[train_pos], sel_y[train_pos])
val_ds   = MappedSubset(base_val,   sel_idx[val_pos],   sel_y[val_pos])

train_loader = DataLoader(train_ds, batch_size=cfg.batch_size, shuffle=True, num_workers=0, pin_memory=False, drop_last=True)
val_loader   = DataLoader(val_ds, batch_size=cfg.batch_size, shuffle=False, num_workers=0, pin_memory=False)

xb, yb = next(iter(train_loader))
print("Sanity labels unique:", torch.unique(yb))
assert yb.min() >= 0 and yb.max() <= 3


Distribuição antes do split:
BMW_3Series 43
BMW_5Series 34
BMW_7Series_or_Fallback 42
undefined 200
Sanity labels unique: tensor([0, 1, 2, 3])


## Model

In [ ]:
model = timm.create_model(cfg.model_name, pretrained=True, num_classes=4).to(device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/86.5M [00:00<?, ?B/s]

In [ ]:
train_labels_all = sel_y[train_pos]
weights = compute_class_weight(
    class_weight="balanced",
    classes=np.array([0,1,2,3]),
    y=train_labels_all
)
weights = torch.tensor(weights, dtype=torch.float32).to(device)

criterion = nn.CrossEntropyLoss(weight=weights, label_smoothing=0.05)


## Focal Loss

In [ ]:
train_labels_all = sel_y[train_pos]
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.array([0,1,2,3]),
    y=train_labels_all
)
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

class FocalLossMulti(nn.Module):
    """
    Focal Loss para multi-classe:
    - suporta targets hard (shape [B]) ou soft (shape [B, C]) (ex.: mixup).
    - suporta class weights (shape [C]).
    """
    def __init__(self, gamma=2.0, weight=None, eps=1e-7):
        super().__init__()
        self.gamma = gamma
        self.register_buffer("weight", weight if weight is not None else None)
        self.eps = eps

    def forward(self, logits, targets):
        log_probs = torch.log_softmax(logits, dim=1)
        probs = torch.exp(log_probs)

        if targets.dim() == 1:
            targets_oh = torch.zeros_like(logits).scatter_(1, targets.unsqueeze(1), 1.0)
        else:
            targets_oh = targets

        pt = (probs * targets_oh).sum(dim=1).clamp(min=self.eps, max=1.0)

        ce = -(targets_oh * log_probs).sum(dim=1)

        if self.weight is not None:
            w = (targets_oh * self.weight.unsqueeze(0)).sum(dim=1)
            ce = ce * w

        loss = ((1.0 - pt) ** self.gamma) * ce
        return loss.mean()

FOCAL_GAMMA = getattr(cfg, "focal_gamma", 2.0)
criterion = FocalLossMulti(gamma=FOCAL_GAMMA, weight=class_weights)

In [ ]:
mixup_fn = None

USE_MIXUP = getattr(cfg, "use_mixup", True)
MIXUP_A   = getattr(cfg, "mixup_alpha", 0.2)

if USE_MIXUP:
    from timm.data import Mixup
    mixup_fn = Mixup(
        mixup_alpha=MIXUP_A,
        cutmix_alpha=0.0,
        label_smoothing=0.0,
        num_classes=4
    )

## Train

In [ ]:
scaler = torch.cuda.amp.GradScaler(enabled=(device=="cuda"))

def run_epoch(model, loader, train=True):
    """
    Assinatura fixa:
      - se train=False, passe optimizer=None
    """
    model.train(train)
    losses = []
    all_probs = []
    all_targets = []

    for x, y in loader:
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        if train and mixup_fn is not None:
            x, y_mix = mixup_fn(x, y)  
        else:
            y_mix = y  

        with torch.cuda.amp.autocast(enabled=(device=="cuda")):
            logits = model(x)
            loss = criterion(logits, y_mix)

        if train:
            optimizer.zero_grad(set_to_none=True)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

        losses.append(loss.item())

        probs = torch.softmax(logits.detach(), dim=1).cpu().numpy()
        all_probs.append(probs)
        all_targets.append(y.detach().cpu().numpy())

    all_probs = np.concatenate(all_probs, axis=0)
    all_targets = np.concatenate(all_targets, axis=0)
    preds = all_probs.argmax(axis=1)

    acc = accuracy_score(all_targets, preds)
    f1  = f1_score(all_targets, preds, average="macro")
    return float(np.mean(losses)), acc, f1


def freeze_backbone(model):
    for p in model.parameters():
        p.requires_grad = False
    for p in model.get_classifier().parameters():
        p.requires_grad = True

def unfreeze_all(model):
    for p in model.parameters():
        p.requires_grad = True

def run_epoch2(model, loader, criterion, optimizer, train: bool):
    """
    Assinatura fixa:
      - se train=False, passe optimizer=None
    """
    model.train(train)
    losses = []
    all_probs = []
    all_targets = []

    for x, y in loader:
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        if train and mixup_fn is not None:
            x, y_mix = mixup_fn(x, y)  
        else:
            y_mix = y  

        with torch.cuda.amp.autocast(enabled=(device=="cuda")):
            logits = model(x)
            loss = criterion(logits, y_mix)

        if train:
            optimizer.zero_grad(set_to_none=True)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

        losses.append(loss.item())

        probs = torch.softmax(logits.detach(), dim=1).cpu().numpy()
        all_probs.append(probs)
        all_targets.append(y.detach().cpu().numpy())

    all_probs = np.concatenate(all_probs, axis=0)
    all_targets = np.concatenate(all_targets, axis=0)
    preds = all_probs.argmax(axis=1)

    acc = accuracy_score(all_targets, preds)
    f1  = f1_score(all_targets, preds, average="macro")
    return float(np.mean(losses)), acc, f1

/tmp/ipython-input-3841390307.py:1: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device=="cuda"))


In [ ]:
def train_eval_short(model_name, lr_full, weight_decay, focal_gamma,
                     epochs_head=2, epochs_full=4, patience=2):
    model = timm.create_model(model_name, pretrained=True, num_classes=4).to(device)

    criterion = FocalLossMulti(gamma=focal_gamma, weight=class_weights)

    freeze_backbone(model)
    opt = torch.optim.AdamW(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=getattr(cfg, "lr_head", 5e-4),
        weight_decay=weight_decay
    )

    best = -1
    bad = 0
    for _ in range(max(1, epochs_head)):
        _ = run_epoch2(model, train_loader, criterion, optimizer=opt, train=True)
        _, _, f1v = run_epoch2(model, val_loader, criterion, optimizer=None, train=False)
        if f1v > best:
            best = f1v; bad = 0
        else:
            bad += 1
            if bad >= patience:
                break

    unfreeze_all(model)
    opt = torch.optim.AdamW(model.parameters(), lr=lr_full, weight_decay=weight_decay)

    best = -1
    bad = 0
    for _ in range(max(1, epochs_full)):
        _ = run_epoch2(model, train_loader, criterion, optimizer=opt, train=True)
        _, _, f1v = run_epoch2(model, val_loader, criterion, optimizer=None, train=False)
        if f1v > best:
            best = f1v; bad = 0
        else:
            bad += 1
            if bad >= patience:
                break

    return best


if getattr(cfg, "do_hparam_search", True):
    search_space = {
        "lr_full":      [5e-5, 1e-4, 2e-4],
        "weight_decay": [1e-5, 1e-4, 5e-4],
        "focal_gamma":  [1.5, 2.0, 2.5],
    }

    results = []
    trial = 0

    for lr in search_space["lr_full"]:
        for wd in search_space["weight_decay"]:
            for g in search_space["focal_gamma"]:
                trial += 1
                f1v = train_eval_short(
                    model_name=cfg.model_name,
                    lr_full=lr,
                    weight_decay=wd,
                    focal_gamma=g,
                    epochs_head=getattr(cfg, "hs_epochs_head", 2),
                    epochs_full=getattr(cfg, "hs_epochs_full", 4),
                    patience=getattr(cfg, "hs_patience", 2)
                )
                results.append({"trial": trial, "lr_full": lr, "weight_decay": wd, "focal_gamma": g, "macro_f1": f1v})
                print(f"trial {trial:02d} | lr={lr:.1e} wd={wd:.1e} gamma={g:.1f} -> macroF1={f1v:.3f}")

    results_sorted = sorted(results, key=lambda x: x["macro_f1"], reverse=True)
    best = results_sorted[0]
    print("\nTOP-5:")
    for r in results_sorted[:5]:
        print(r)

    cfg.lr_full = best["lr_full"]
    cfg.weight_decay = best["weight_decay"]
    cfg.focal_gamma = best["focal_gamma"]

    print("\n Selected hparams:")
    print("lr_full     =", cfg.lr_full)
    print("weight_decay=", cfg.weight_decay)
    print("focal_gamma =", cfg.focal_gamma)


/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):
/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):
/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):
/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):
/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.auto

trial 01 | lr=5.0e-05 wd=1.0e-05 gamma=1.5 -> macroF1=0.283


/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):
/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):
/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):
/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):
/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.auto

trial 02 | lr=5.0e-05 wd=1.0e-05 gamma=2.0 -> macroF1=0.328


/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):
/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):
/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):
/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):
/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.auto

...

trial 25 | lr=2.0e-04 wd=5.0e-04 gamma=1.5 -> macroF1=0.651


/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):
/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):
/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):
/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):
/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.auto

trial 26 | lr=2.0e-04 wd=5.0e-04 gamma=2.0 -> macroF1=0.462


/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):
/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):
/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):
/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):
/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.auto

trial 27 | lr=2.0e-04 wd=5.0e-04 gamma=2.5 -> macroF1=0.521

TOP-5:
{'trial': 25, 'lr_full': 0.0002, 'weight_decay': 0.0005, 'focal_gamma': 1.5, 'macro_f1': 0.6512445887445887}
{'trial': 24, 'lr_full': 0.0002, 'weight_decay': 0.0001, 'focal_gamma': 2.5, 'macro_f1': 0.6179246635962827}
{'trial': 23, 'lr_full': 0.0002, 'weight_decay': 0.0001, 'focal_gamma': 2.0, 'macro_f1': 0.5712438423645321}
{'trial': 27, 'lr_full': 0.0002, 'weight_decay': 0.0005, 'focal_gamma': 2.5, 'macro_f1': 0.5212923500796418}
{'trial': 22, 'lr_full': 0.0002, 'weight_decay': 0.0001, 'focal_gamma': 1.5, 'macro_f1': 0.5199967083607637}

✅ Selected hparams:
lr_full     = 0.0002
weight_decay= 0.0005
focal_gamma = 1.5


## Fine Turning

In [ ]:
model = timm.create_model(cfg.model_name, pretrained=True, num_classes=4).to(device)

criterion = FocalLossMulti(gamma=getattr(cfg, "focal_gamma", 2.0), weight=class_weights)

EPOCHS_HEAD = getattr(cfg, "epochs_head", 5)
EPOCHS_FULL = getattr(cfg, "epochs_full", 15)
LR_HEAD     = getattr(cfg, "lr_head", 5e-4)
LR_FULL     = getattr(cfg, "lr_full", 1e-4)
WD          = getattr(cfg, "weight_decay", 1e-4)
PATIENCE    = getattr(cfg, "patience", 4)

best_f1 = -1
pat = 0
best_path = "/content/best.pt"

freeze_backbone(model)
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LR_HEAD, weight_decay=WD)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max(1, EPOCHS_HEAD))

print("\n=== Final Phase 1: Head training (freeze) ===")
for epoch in range(EPOCHS_HEAD):
    tr_loss, tr_acc, tr_f1 = run_epoch2(model, train_loader, criterion, optimizer, train=True)
    va_loss, va_acc, va_f1 = run_epoch2(model, val_loader, criterion, None, train=False)
    scheduler.step()

    print(f"[Head] Epoch {epoch+1:02d} | train f1 {tr_f1:.3f} | val f1 {va_f1:.3f}")

    if va_f1 > best_f1:
        best_f1 = va_f1
        pat = 0
        torch.save(model.state_dict(), best_path)
    else:
        pat += 1
        if pat >= PATIENCE:
            print("Early stopping (head).")
            break

model.load_state_dict(torch.load(best_path, map_location=device))
unfreeze_all(model)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR_FULL, weight_decay=WD)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max(1, EPOCHS_FULL))
pat = 0

print("\n=== Final Phase 2: Full fine-tuning (unfreeze) ===")
for epoch in range(EPOCHS_FULL):
    tr_loss, tr_acc, tr_f1 = run_epoch2(model, train_loader, criterion, optimizer=optimizer, train=True)
    va_loss, va_acc, va_f1 = run_epoch2(model, val_loader, criterion, optimizer=None, train=False)
    scheduler.step()

    print(f"[Full] Epoch {epoch+1:02d} | train f1 {tr_f1:.3f} | val f1 {va_f1:.3f}")

    if va_f1 > best_f1:
        best_f1 = va_f1
        pat = 0
        torch.save(model.state_dict(), best_path)
    else:
        pat += 1
        if pat >= PATIENCE:
            print("Early stopping (full).")
            break

print("\nBest val macro-F1:", best_f1)


=== Final Phase 1: Head training (freeze) ===


/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):
/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):


[Head] Epoch 01 | train f1 0.151 | val f1 0.108


/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):
/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):


[Head] Epoch 02 | train f1 0.178 | val f1 0.085


...

[Full] Epoch 07 | train f1 0.626 | val f1 0.550


/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):
/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):


[Full] Epoch 08 | train f1 0.451 | val f1 0.551


/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):
/tmp/ipython-input-3841390307.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):


[Full] Epoch 09 | train f1 0.526 | val f1 0.556
Early stopping (full).

Best val macro-F1: 0.5813156906906907


## Evaluat

In [ ]:
!pip -q install onnx onnxruntime onnxscript

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 689.1/689.1 kB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.3/159.3 kB 17.1 MB/s eta 0:00:00


In [ ]:
model.load_state_dict(torch.load(best_path, map_location=device))
model.eval()

def predict_probs(model, loader):
    model.eval()
    probs_all, y_all = [], []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device, non_blocking=True)
            logits = model(x)
            probs = torch.softmax(logits, dim=1).cpu().numpy()
            probs_all.append(probs)
            y_all.append(y.numpy())
    return np.concatenate(probs_all, axis=0), np.concatenate(y_all, axis=0)

va_probs, va_targets = predict_probs(model, val_loader)

print("\nSem rejeição:")
pred = va_probs.argmax(axis=1)
print(classification_report(va_targets, pred, target_names=[idx_to_name[i] for i in range(4)]))

def predict_with_rejection(probs, thr):
    conf = probs.max(axis=1)
    pred = probs.argmax(axis=1)
    pred_rej = pred.copy()
    pred_rej[conf < thr] = 3
    return pred, pred_rej, conf

ths = np.linspace(getattr(cfg,"thr_min",0.2), getattr(cfg,"thr_max",0.9), getattr(cfg,"thr_steps",29))
best_t, best_macro = None, -1
for t in ths:
    _, pr, _ = predict_with_rejection(va_probs, t)
    m = f1_score(va_targets, pr, average="macro")
    if m > best_macro:
        best_macro = m
        best_t = float(t)

print(f"\nBest threshold (macro-F1): {best_t:.3f} | macro-F1: {best_macro:.3f}")

_, pred_rej, _ = predict_with_rejection(va_probs, best_t)
print("\nCom rejeição (auto threshold):", best_t)
print(classification_report(va_targets, pred_rej, target_names=[idx_to_name[i] for i in range(4)]))

onnx_path = "/content/car_classifier.onnx"
dummy = torch.randn(1, 3, cfg.img_size, cfg.img_size).to(device)

torch.onnx.export(
    model,
    dummy,
    onnx_path,
    input_names=["input"],
    output_names=["logits"],
    dynamic_axes={"input": {0: "batch"}, "logits": {0: "batch"}},
    opset_version=17
)
print("\nONNX exportado:", onnx_path)

def bench_latency(model, device, n_warmup=20, n_iters=100):
    model.eval()
    x = torch.randn(1, 3, cfg.img_size, cfg.img_size).to(device)
    with torch.no_grad():
        for _ in range(n_warmup):
            _ = model(x)
        if device == "cuda":
            torch.cuda.synchronize()
        t0 = time.time()
        for _ in range(n_iters):
            _ = model(x)
        if device == "cuda":
            torch.cuda.synchronize()
        dt = (time.time() - t0) / n_iters
    print(f"Latency médio: {dt*1000:.2f} ms | device={device} | batch=1 | iters={n_iters}")

bench_latency(model, device)


Sem rejeição:
                         precision    recall  f1-score   support

            BMW_3Series       0.42      0.77      0.54        13
            BMW_5Series       0.62      0.50      0.56        10
BMW_7Series_or_Fallback       0.47      0.69      0.56        13
              undefined       0.78      0.58      0.67        60

               accuracy                           0.61        96
              macro avg       0.57      0.64      0.58        96
           weighted avg       0.67      0.61      0.62        96


Best threshold (macro-F1): 0.775 | macro-F1: 0.616

Com rejeição (auto threshold): 0.7749999999999999
                         precision    recall  f1-score   support

            BMW_3Series       0.48      0.77      0.59        13
            BMW_5Series       0.62      0.50      0.56        10
BMW_7Series_or_Fallback       0.57      0.62      0.59        13
              undefined       0.77      0.68      0.73        60

               accuracy         

/tmp/ipython-input-2579829551.py:48: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W0217 18:30:20.699000 1657 torch/onnx/_internal/exporter/_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 17 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features
W0217 18:30:21.831000 1657 torch/onnx/_internal/exporter/_schemas.py:455] Missing annotation for parameter 'input' from (input, boxes, output_size: 'Sequence[int]', spatial_scale: 'float' = 1.0, sampling_ratio: 'int' = -1, al

[torch.onnx] Obtain model graph for `EfficientNet([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `EfficientNet([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...


[torch.onnx] Translate the graph into ONNX... ✅


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/onnxscript/version_converter/__init__.py", line 120, in call
    converted_proto = _c_api_utils.call_onnx_api(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/onnxscript/version_converter/_c_api_utils.py", line 65, in call_onnx_api
    result = func(proto)
             ^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/onnxscript/version_converter/__init__.py", line 115, in _partial_convert_version
    return onnx.version_converter.convert_version(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/onnx/version_converter.py", line 39, in convert_version
    converted_model_str = C.convert_version(model_str, target_version)
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: /github/workspace/onnx/version_converter/BaseConverter.h:65: adapter_lookup: Assertion `false`

Applied 225 of general pattern rewrite rules.

ONNX exportado: /content/car_classifier.onnx
Latency médio: 19.09 ms | device=cuda | batch=1 | iters=100


## TTA + Threshold

In [ ]:
import numpy as np

def predict_probs_tta_flip(model, loader):
    """
    TTA simples: média das probabilidades
      - original
      - flip horizontal
    """
    model.eval()
    probs_all, y_all = [], []

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device, non_blocking=True)

            logits1 = model(x)
            p1 = torch.softmax(logits1, dim=1)

            x_flip = torch.flip(x, dims=[3])  
            logits2 = model(x_flip)
            p2 = torch.softmax(logits2, dim=1)

            p = 0.5 * (p1 + p2)

            probs_all.append(p.cpu().numpy())
            y_all.append(y.numpy())

    return np.concatenate(probs_all, axis=0), np.concatenate(y_all, axis=0)

In [ ]:
from sklearn.metrics import f1_score, classification_report

def apply_classwise_rejection(probs, thr_by_class, undefined_id=3):
    """
    probs: [N,C]
    thr_by_class: dict {class_id: thr}
    regra:
      pred = argmax
      se conf(pred) < thr[pred] -> undefined
    """
    conf = probs.max(axis=1)
    pred = probs.argmax(axis=1)
    pred_rej = pred.copy()

    for i in range(len(pred)):
        c = int(pred[i])
        if c == undefined_id:
            continue
        thr = thr_by_class.get(c, 0.5)
        if conf[i] < thr:
            pred_rej[i] = undefined_id

    return pred, pred_rej, conf


def search_thresholds_per_class(probs, y_true, thr_grid=None, undefined_id=3, class_ids=(0,1,2)):
    """
    Busca thresholds por classe maximizando macro-F1 (com rejeição classwise).
    Estratégia leve e efetiva:
      - varre um threshold por vez mantendo os outros fixos
      - repete por 2-3 rodadas
    """
    if thr_grid is None:
        thr_grid = np.linspace(0.30, 0.95, 14)

    thr_by = {c: 0.6 for c in class_ids}

    def score(thr_by_local):
        _, pred_rej, _ = apply_classwise_rejection(probs, thr_by_local, undefined_id=undefined_id)
        return f1_score(y_true, pred_rej, average="macro")

    best_score = score(thr_by)

    for _round in range(3):
        improved = False
        for c in class_ids:
            cur_best_t = thr_by[c]
            cur_best_s = best_score

            for t in thr_grid:
                trial = dict(thr_by)
                trial[c] = float(t)
                s = score(trial)
                if s > cur_best_s:
                    cur_best_s = s
                    cur_best_t = float(t)

            if cur_best_s > best_score:
                thr_by[c] = cur_best_t
                best_score = cur_best_s
                improved = True

        if not improved:
            break

    return thr_by, best_score

In [ ]:

model.load_state_dict(torch.load(best_path, map_location=device))
model.eval()

va_probs_tta, va_targets = predict_probs_tta_flip(model, val_loader)

print("\n=== TTA (flip) | Sem rejeição ===")
pred = va_probs_tta.argmax(axis=1)
print(classification_report(va_targets, pred, target_names=[idx_to_name[i] for i in range(4)]))

thr_grid = np.linspace(0.35, 0.95, 13)
thr_by_class, best_macro = search_thresholds_per_class(
    va_probs_tta, va_targets, thr_grid=thr_grid, undefined_id=3, class_ids=(0,1,2)
)

print("\nBest thresholds por classe:", thr_by_class)
print("Best macro-F1 (classwise rejection):", best_macro)

_, pred_rej, _ = apply_classwise_rejection(va_probs_tta, thr_by_class, undefined_id=3)

print("\n=== TTA (flip) | Com rejeição (threshold por classe) ===")
print(classification_report(va_targets, pred_rej, target_names=[idx_to_name[i] for i in range(4)]))


=== TTA (flip) | Sem rejeição ===
                         precision    recall  f1-score   support

            BMW_3Series       0.40      0.62      0.48        13
            BMW_5Series       0.46      0.60      0.52        10
BMW_7Series_or_Fallback       0.50      0.77      0.61        13
              undefined       0.74      0.53      0.62        60

               accuracy                           0.58        96
              macro avg       0.53      0.63      0.56        96
           weighted avg       0.64      0.58      0.59        96


Best thresholds por classe: {0: 0.8999999999999999, 1: 0.75, 2: 0.8999999999999999}
Best macro-F1 (classwise rejection): 0.7028518356643356

=== TTA (flip) | Com rejeição (threshold por classe) ===
                         precision    recall  f1-score   support

            BMW_3Series       0.62      0.62      0.62        13
            BMW_5Series       0.83      0.50      0.62        10
BMW_7Series_or_Fallback       0.89      0.62   